In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from scipy import signal 
from tqdm import tqdm 
from numpy.fft import fft,fftshift
import random 
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
train=pd.read_csv('./open/train_features.csv')
train_label=pd.read_csv('./open/train_labels.csv')
test=pd.read_csv('./open/test_features.csv')
submission=pd.read_csv('./open/sample_submission.csv')
pd.options.display.max_columns=50

In [4]:
train['acc_Energy']=(train['acc_x']**2+train['acc_y']**2+train['acc_z']**2)**(1/3)
test['acc_Energy']=(test['acc_x']**2+test['acc_y']**2+test['acc_z']**2)**(1/3)

train['gy_Energy']=(train['gy_x']**2+train['gy_y']**2+train['gy_z']**2)**(1/3)
test['gy_Energy']=(test['gy_x']**2+test['gy_y']**2+test['gy_z']**2)**(1/3)

train['gy_acc_Energy']=((train['gy_x']-train['acc_x'])**2+(train['gy_y']-train['acc_y'])**2+(train['gy_z']-train['acc_z'])**2)**(1/3)
test['gy_acc_Energy']=((test['gy_x']-test['acc_x'])**2+(test['gy_y']-test['acc_y'])**2+(test['gy_z']-test['acc_z'])**2)**(1/3)

In [5]:
dt=0.02
def jerk_signal(signal):
    return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [6]:
train_dt=list()
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    train_dt.append(temp )

100%|██████████| 3125/3125 [00:42<00:00, 73.62it/s]


In [7]:
test_dt=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    test_dt.append(temp)

100%|██████████| 782/782 [00:08<00:00, 89.38it/s]


In [8]:
from scipy import fftpack
from numpy.fft import *

def fourier_transform_one_signal(t_signal):
    complex_f_signal= fftpack.fft(t_signal)
    amplitude_f_signal=np.abs(complex_f_signal)
    return amplitude_f_signal

In [9]:
train=pd.concat(train_dt)

In [10]:
train.info()
fft=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for i in train.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft.append(temp)
train=pd.concat(fft)
test=pd.concat(test_dt)

  1%|          | 31/3125 [00:00<00:10, 307.15it/s]<class 'pandas.core.frame.DataFrame'>
Int64Index: 1875000 entries, 0 to 1874999
Data columns (total 20 columns):
 #   Column            Dtype  
---  ------            -----  
 0   id                int64  
 1   time              int64  
 2   acc_x             float64
 3   acc_y             float64
 4   acc_z             float64
 5   gy_x              float64
 6   gy_y              float64
 7   gy_z              float64
 8   acc_Energy        float64
 9   gy_Energy         float64
 10  gy_acc_Energy     float64
 11  acc_x_dt          float64
 12  acc_y_dt          float64
 13  acc_z_dt          float64
 14  gy_x_dt           float64
 15  gy_y_dt           float64
 16  gy_z_dt           float64
 17  acc_Energy_dt     float64
 18  gy_Energy_dt      float64
 19  gy_acc_Energy_dt  float64
dtypes: float64(18), int64(2)
memory usage: 300.4 MB
100%|██████████| 3125/3125 [00:09<00:00, 316.93it/s]


In [11]:
fft_t=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for i in test.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft_t.append(temp)
test=pd.concat(fft_t)

100%|██████████| 782/782 [00:01<00:00, 529.59it/s]


In [12]:
col=train.columns
train_s=train.copy()
test_s=test.copy()
from sklearn.preprocessing import StandardScaler,MinMaxScaler

scaler=StandardScaler()
train_s.iloc[:,2:]=scaler.fit_transform(train_s.iloc[:,2:])
train_sc=pd.DataFrame(data=train_s,columns=col)
test_s.iloc[:,2:]=scaler.transform(test_s.iloc[:,2:])
test_sc=pd.DataFrame(test_s,columns=col)

In [13]:
train_sc

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_Energy,gy_Energy,gy_acc_Energy,acc_x_dt,acc_y_dt,acc_z_dt,gy_x_dt,gy_y_dt,gy_z_dt,acc_Energy_dt,gy_Energy_dt,gy_acc_Energy_dt
0,0,0,27.356382,8.807207,19.465910,0.376992,0.869226,0.150423,0.495681,-0.272719,-0.276391,0.000027,0.000298,-0.000433,0.000347,0.000373,0.000273,0.000101,0.001505,0.001501
1,0,1,-0.054866,0.833464,0.820412,-0.282128,-0.093560,0.011266,0.742974,-0.236152,-0.240632,0.416836,-0.118821,-0.255054,0.032738,-0.349095,0.377085,0.564992,0.166566,0.162871
2,0,2,0.024046,0.315921,0.081086,-0.182551,-0.053585,-0.003708,0.819822,-0.169815,-0.173080,0.086405,0.023750,-0.531727,-0.141582,-0.202368,-0.004887,0.175645,0.300944,0.306341
3,0,3,0.065632,0.117634,-0.040874,-0.194863,0.154242,0.005408,0.785669,-0.035229,-0.040560,-0.058780,-0.213920,0.285459,0.229520,-0.385106,-0.135647,-0.077915,0.609008,0.599518
4,0,4,0.151477,0.300751,0.317742,-0.350724,0.494539,0.154354,0.791528,0.021954,0.016872,0.039823,0.259227,-0.055206,0.057320,-0.174917,-0.028047,0.013483,0.259626,0.260669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1874995,3124,595,0.365037,0.011656,0.845701,0.080839,0.350395,0.112282,-0.138940,0.829394,0.823900,0.151679,0.037205,0.119409,-0.108728,-0.027804,-0.009085,-0.142794,0.063329,0.063674
1874996,3124,596,10.220817,5.476964,7.441373,3.605246,16.530576,11.843241,-0.167578,0.814816,0.809618,0.150658,-0.000363,0.265559,-0.027936,0.090560,-0.018412,-0.065316,-0.064300,-0.062949
1874997,3124,597,0.386337,0.177768,-0.080193,-0.192468,-0.033904,-0.227861,-0.151875,0.802027,0.797338,0.093524,-0.049283,0.260884,0.082744,0.123264,-0.152712,0.035970,-0.056225,-0.053918
1874998,3124,598,0.728823,0.014037,0.350745,0.136284,1.281790,0.403540,-0.175811,0.801880,0.797431,0.174681,-0.096564,0.071332,0.153722,-0.014412,-0.049662,-0.054574,0.000843,0.001922


In [14]:
import tensorflow as tf 
import tensorflow.keras as keras
import tensorflow_addons as tfa 
from keras.models import Sequential
from keras.layers import Dense,LSTM,Bidirectional,Dropout
from keras.layers.convolutional import Conv1D,MaxPooling1D
from keras.utils import to_categorical
from keras import backend as K 
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from sklearn.model_selection import KFold,StratifiedKFold
from numpy.random import seed 
import keras

In [15]:
train_sc.shape
X=np.array(train_sc.iloc[:,2:]).reshape(3125,600,-1)
X.shape

(3125, 600, 18)

In [16]:
test_x=np.array(test_sc.iloc[:,2:]).reshape(782, 600, -1)
test_x.shape

(782, 600, 18)

In [17]:
print(train_label)
y = train_label['label'].values
y = tf.keras.utils.to_categorical(train_label['label']) 
y.shape

        id  label                          label_desc
0        0     37           Shoulder Press (dumbbell)
1        1     26                        Non-Exercise
2        2      3                  Biceps Curl (band)
3        3     26                        Non-Exercise
4        4     26                        Non-Exercise
...    ...    ...                                 ...
3120  3120     26                        Non-Exercise
3121  3121     26                        Non-Exercise
3122  3122     15  Dynamic Stretch (at your own pace)
3123  3123     26                        Non-Exercise
3124  3124      2                          Bicep Curl

[3125 rows x 3 columns]


(3125, 61)

In [18]:
def cnn_model(input_shape,classes):
    seed(2021)
    tf.random.set_seed(20201)
    input_layer = keras.layers.Input(input_shape)
    conv1 = keras.layers.Conv1D(filters=128, kernel_size=9, padding='same')(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation(activation='relu')(conv1)
    conv1 = keras.layers.Dropout(rate=0.3)(conv1)

    conv2 = keras.layers.Conv1D(filters=256, kernel_size=6, padding='same')(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)
    conv2 = keras.layers.Dropout(rate=0.4)(conv2)
    
    conv3 = keras.layers.Conv1D(128, kernel_size=3,padding='same')(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation('relu')(conv3)
    conv3 = keras.layers.Dropout(rate=0.5)(conv3)
    
    gap = keras.layers.GlobalAveragePooling1D()(conv3)
    
    output_layer = keras.layers.Dense(classes, activation='softmax')(gap)
    
    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(), 
        metrics=['accuracy'])
    
    return model

In [19]:
skf = StratifiedKFold(n_splits = 10, random_state = 2021, shuffle = True)
reLR = ReduceLROnPlateau(patience = 4,verbose = 1,factor = 0.5) 
es =EarlyStopping(monitor='val_loss', patience=8, mode='min')

accuracy = []
losss=[]
models=[]

for i, (train, validation) in enumerate(skf.split(X, y.argmax(1))) :
    mc = ModelCheckpoint(f'./open/cv_study{i + 1}.h5',save_best_only=True, verbose=0, monitor = 'val_loss', mode = 'min', save_weights_only=True)
    print("-" * 20 +"Fold_"+str(i+1)+ "-" * 20)
    model = cnn_model((600,18),61)
    history = model.fit(X[train], y[train], epochs = 100, validation_data= (X[validation], y[validation]), 
                        verbose=1,batch_size=64,callbacks=[es,mc,reLR])
    model.load_weights(f'./open/cv_study{i + 1}.h5')
    
    k_accuracy = '%.4f' % (model.evaluate(X[validation], y[validation])[1])
    k_loss = '%.4f' % (model.evaluate(X[validation], y[validation])[0])
    
    accuracy.append(k_accuracy)
    losss.append(k_loss)
    models.append(model)

print('\nK-fold cross validation Auc: {}'.format(accuracy))
print('\nK-fold cross validation loss: {}'.format(losss))

9ms/step - loss: 0.5786 - accuracy: 0.8303 - val_loss: 0.7774 - val_accuracy: 0.8051
Epoch 19/100
44/44 [==============================] - 2s 54ms/step - loss: 0.5427 - accuracy: 0.8495 - val_loss: 0.7549 - val_accuracy: 0.7955
Epoch 20/100
44/44 [==============================] - 2s 53ms/step - loss: 0.5159 - accuracy: 0.8511 - val_loss: 0.7109 - val_accuracy: 0.7923
Epoch 21/100
44/44 [==============================] - 2s 50ms/step - loss: 0.4848 - accuracy: 0.8521 - val_loss: 0.7291 - val_accuracy: 0.7764
Epoch 22/100
44/44 [==============================] - 2s 52ms/step - loss: 0.4892 - accuracy: 0.8524 - val_loss: 0.7152 - val_accuracy: 0.7955
Epoch 23/100
44/44 [==============================] - 2s 49ms/step - loss: 0.4858 - accuracy: 0.8504 - val_loss: 0.6632 - val_accuracy: 0.8083
Epoch 24/100
44/44 [==============================] - 2s 50ms/step - loss: 0.4387 - accuracy: 0.8708 - val_loss: 0.6621 - val_accuracy: 0.8051
Epoch 25/100
44/44 [==============================] - 2s 

In [20]:
test_X=np.array(test_sc.iloc[:,2:]).reshape(782, 600, -1)
test_X.shape

(782, 600, 18)

In [21]:
preds = []
for model in models:
    pred = model.predict(test_X)
    preds.append(pred)
pred = np.mean(preds, axis=0)
pred

array([[4.3955456e-06, 1.0051062e-06, 3.4406258e-07, ..., 4.3151681e-03,
        5.7393358e-06, 3.1050745e-06],
       [5.7655328e-04, 2.7663316e-05, 5.3001568e-05, ..., 3.3672854e-06,
        2.5347626e-05, 1.6828923e-05],
       [6.7229557e-04, 2.6267871e-02, 4.5714414e-06, ..., 3.5736561e-04,
        7.2013624e-03, 3.5649729e-03],
       ...,
       [4.3212710e-04, 5.0058084e-06, 5.8348714e-06, ..., 1.5295202e-05,
        7.4118151e-07, 1.2497038e-03],
       [6.0677394e-06, 6.0142553e-04, 4.4212410e-07, ..., 1.2092964e-07,
        3.8138357e-06, 1.2453638e-08],
       [7.6904784e-05, 4.1955818e-06, 8.4090186e-07, ..., 2.3960423e-04,
        1.2979629e-06, 2.9697843e-04]], dtype=float32)

In [22]:
submission=pd.read_csv('./open/sample_submission.csv')
submission.iloc[:,1:]=pred
submission

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,...,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,0.000004,0.000001,3.440626e-07,1.768032e-08,1.430821e-04,5.899136e-07,1.405213e-04,0.000013,5.970440e-08,0.007118,6.998050e-02,4.721001e-01,4.764451e-04,4.223833e-01,1.692203e-03,8.108493e-06,1.865974e-06,2.750837e-07,6.628549e-07,5.420466e-06,3.000905e-06,2.785819e-06,1.190323e-06,0.011513,...,8.268285e-04,6.636359e-04,1.423408e-04,1.831121e-05,7.080535e-08,1.448755e-07,0.000945,0.000090,3.519620e-05,1.346798e-03,1.241451e-05,1.571495e-07,4.367356e-06,3.727425e-06,3.882373e-08,7.158775e-04,1.799114e-04,3.515418e-06,5.159307e-06,1.093319e-07,2.205167e-05,3.069901e-09,4.315168e-03,5.739336e-06,3.105074e-06
1,3126,0.000577,0.000028,5.300157e-05,1.959058e-03,5.176046e-05,5.458228e-04,6.400153e-06,0.000076,3.841904e-06,0.000022,1.020382e-05,6.470441e-07,6.671296e-08,1.219125e-06,1.682519e-06,2.643538e-04,8.169426e-06,1.423647e-05,1.677734e-05,3.802030e-06,2.032817e-05,1.472189e-04,3.454946e-03,0.000140,...,1.630262e-04,3.529139e-06,3.150341e-06,7.636694e-08,4.029530e-04,3.600028e-04,0.000010,0.000009,8.909231e-07,1.910726e-06,1.017850e-05,3.355664e-05,1.786967e-04,2.073932e-03,7.157866e-04,6.616122e-06,6.415556e-08,1.005095e-04,2.985222e-04,9.486639e-05,1.273018e-05,1.075205e-04,3.367285e-06,2.534763e-05,1.682892e-05
2,3127,0.000672,0.026268,4.571441e-06,2.903764e-06,3.132365e-05,3.466806e-04,3.492374e-02,0.000149,2.000439e-05,0.000369,1.338044e-03,3.982928e-04,5.649381e-05,4.395702e-03,1.394045e-02,8.687613e-04,1.727450e-04,4.597991e-04,2.301811e-05,2.904742e-04,3.732721e-04,8.402640e-05,1.085599e-07,0.000250,...,1.453728e-03,8.734466e-02,8.290728e-03,1.459409e-04,3.829252e-04,1.235150e-04,0.028401,0.231138,1.160992e-02,4.729814e-01,9.531206e-04,7.486226e-04,1.829613e-03,2.491417e-03,7.356184e-07,9.112000e-05,2.348217e-05,1.185400e-06,9.130159e-04,8.107526e-07,1.915725e-03,6.510400e-08,3.573656e-04,7.201362e-03,3.564973e-03
3,3128,0.000444,0.000009,9.982874e-05,2.026598e-04,5.966680e-06,4.726128e-05,4.240780e-06,0.000141,1.050581e-03,0.000019,2.445121e-04,7.195385e-05,3.480275e-06,2.057023e-06,2.487789e-05,2.828537e-04,3.279070e-07,3.549706e-06,2.182547e-06,1.598718e-06,3.212181e-06,6.472810e-05,3.767232e-05,0.000081,...,2.424678e-04,1.056190e-06,5.237694e-05,9.614967e-07,7.052824e-06,4.694936e-06,0.000014,0.000024,1.516358e-05,2.212631e-05,7.816576e-06,3.818717e-05,2.224833e-03,6.101608e-03,8.398696e-04,1.489779e-04,9.851968e-06,2.031352e-05,2.558382e-05,3.767787e-05,1.848217e-06,3.832962e-05,1.474924e-05,3.222499e-06,6.864996e-03
4,3129,0.001811,0.000027,1.599152e-06,5.231936e-04,4.356049e-04,3.635793e-05,9.546840e-07,0.000003,1.444059e-05,0.000040,2.423781e-05,6.675769e-06,8.358064e-07,4.521470e-07,2.454368e-05,4.665540e-04,9.037330e-06,1.200840e-05,1.359197e-04,2.191336e-06,4.354409e-06,7.573354e-06,1.892100e-04,0.000011,...,1.491847e-05,2.577658e-07,2.068980e-05,1.917756e-06,9.711699e-07,6.547080e-06,0.000040,0.000031,7.696641e-08,5.184464e-07,1.530681e-05,2.533567e-06,4.491681e-04,7.765405e-04,4.753325e-04,9.975126e-06,8.568231e-07,1.432784e-05,2.281642e-06,4.715525e-06,1.309887e-06,1.383369e-03,5.538989e-05,2.613220e-06,5.682869e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0.003376,0.000024,9.289932e-07,2.241921e-04,6.610556e-04,7.387674e-05,5.232951e-07,0.000005,4.411094e-05,0.000009,1.146871e-05,5.468364e-07,1.348126e-07,2.086083e-07,4.269572e-06,5.250274e-04,1.108902e-05,8.549371e-06,2.838518e-04,2.184845e-06,2.186964e-06,3.047839e-06,2.014188e-04,0.000004,...,5.442039e-06,1.301118e-07,1.054748e-05,1.026104e-07,3.990949e-06,7.954625e-06,0.000008,0.000003,1.609850e-07,4.242834e-07,1.483794e-05,2.448846e-06,2.470944e-04,4.117446e-04,7.730624e-04,6.550279e-07,7.858764e-08,3.598512e-06,5.697138e-07,1.3

In [23]:
submission.to_csv('sub_kfold_stratified_10_adam_fft_0.5.csv',index=False)